# How climate and envrionmental justice tools are used in research

studies

Elham Ali (Public Environmental Data Project (PEDP))  
October 5, 2025

## Background

When public climate and environmental justice (EJ) evidence is removed, restricted, or altered, a decade of downstream knowledge becomes harder to verify, reproduce, teach, or apply—especially for the communities and decisions that most need it.

This analysis explores how research studies have used five U.S. federal climate and EJ tools, including their topics and use cases.

## Research Questions

This analysis focuses on three main questions:

-   How many research papers or studies have cited or relied on the five U.S. federal climate and environmental justice tools that are threatened, not being maintained, or no longer publicly available?
-   What types of research topics occur most frequently overall and within each tool, and how do these topics vary by citation frequency?
-   In what ways are these tools used in research studies overall and within each tool, and how does their use vary by citation frequency?

## Data Sources

The climate and EJ tools assessed are:

-   CEQ’s **Climate and Economic Justice Screening Tool (CEJST)**
-   EPA’s **EJScreen**
-   USDA Forest Service’s **Climate Risk Viewer (CRV)**
-   FEMA’s **Future Risk Index (FRI)**
-   CDC’s **Environmental Justice Index (EJI)**

All data were collected from **Google Scholar** and last refreshed on **September 15, 2025**.

Raw datasets are stored in the `01_raw_data/` folder. This script cleans and merges them into analysis-ready datasets.

## Cleaning

I first load all required packages for data handling, cleaning, and visualization.

In [ ]:
## Folder structure helpers
library(here)

here() starts at /Users/elhamali/Documents/Data Projects/climate-ej-tools-publications

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ purrr     1.1.0
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Attaching package: 'janitor'

The following objects are masked from 'package:stats':

    chisq.test, fisher.test

Loading required package: grid

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Highcharts (www.highcharts.com) is a Highsoft software product which is
not free for commercial and Governmental use


Attaching package: 'igraph'

The following objects are masked from 'package:lubridate':

    %--%, union

The following objects are masked from 'package:dplyr':

    as_data_frame, groups, union

The following objects are masked from 'package:purrr':

    compose, simplify

The following object is masked from 'package:tidyr':

    crossing

The following object is masked from 'package:tibble':

    as_data_frame

The following objects are masked from 'package:stats':

    decompose, spectrum

The following object is masked from 'package:base':

    union

### Import raw data

All `.csv` files from the `01_raw_data/` folder are imported and saved as `.rds` files in `05_cleaned_datasets/`.

Saving as `.rds` helps preserve structure and improves loading speed later.

In [ ]:
# List all CSV files
csv_files <- list.files(
  path    = here("01_raw_data"),
  pattern = "\\.csv$",
  full.names = TRUE
)

# Read into a list of dataframes
datasets <- map(csv_files, read.csv)
names(datasets) <- tools::file_path_sans_ext(basename(csv_files))

# Save each dataset as .rds in 05_cleaned_datasets/
walk2(
  datasets,
  names(datasets),
  ~ saveRDS(.x, here("05_cleaned_datasets", paste0(.y, ".rds")))
)

### Merge EJScreen datasets

Google Scholar limits exports to 1,000 records per search \[@harzing2025\].  

Because EJScreen returned more than this, I split it into four parts (`ejscreen_1`–`ejscreen_4`). Here, I merge them into a single dataframe.

In [ ]:
# List EJScreen RDS files
ejscreen_files <- here("05_cleaned_datasets") |>
  list.files(pattern = "^ejscreen_\\d+\\.rds$", full.names = TRUE)

# Read, tag, clean variable names
ejscreen_list <- map(ejscreen_files, ~ {
  readRDS(.x) |>
    mutate(source_file = tools::file_path_sans_ext(basename(.x))) |>
    janitor::clean_names()
})

# Merge into one dataframe
ejscreen_all <- bind_rows(ejscreen_list)

# Quick check on dimensions
dim(ejscreen_all)

[1] 1258   27

### Clean and tag all other tools

I apply the same process to the other tools—CEJST, CRV, EJI, and FRI—to ensure consistency. Each dataset is:

-   Standardized to `snake_case` column names
-   Tagged with its source file and tool name
-   Type-converted for appropriate variable formats

In [ ]:
library(dplyr)

# Helper function: read, clean names, add provenance, type-convert
read_clean_tag <- function(path) {
  df <- readRDS(path) |>
    janitor::clean_names()

  source_stem <- tools::file_path_sans_ext(basename(path)) # e.g., "ejscreen_1" or "cejst"

  df |>
    dplyr::mutate(
      source_file = source_stem,
      tool_name   = sub("_\\d+$", "", source_stem) # "ejscreen_1" -> "ejscreen"; "cejst" -> "cejst"
    ) |>
    readr::type_convert(
      col_types = readr::cols(.default = readr::col_guess())
      ) |>
    mutate(across(where(is.character), trimws))
}

# Process other tools
other_tools <- c("cejst", "crv", "eji", "fri")
other_paths <- here("05_cleaned_datasets", paste0(other_tools, ".rds"))

other_list <- other_paths |>
  set_names(~ tools::file_path_sans_ext(basename(.x))) |>
  map(read_clean_tag)

### Final merge and labeling

Finally, I merge all tools into one dataset and standardize metadata. Steps include:

-   Harmonizing date formats

-   Mapping tool codes (e.g., `ejscreen`) to their full names

-   Dropping empty or placeholder columns

In [ ]:
# Helper: coerce anything date-like into Date
to_Date <- function(x) {
  if (inherits(x, "Date")) {
    return(x)
  }
  if (inherits(x, "POSIXt")) {
    return(as_date(x))
  }
  if (is.numeric(x)) {
    return(as_date(as.POSIXct(x, origin = "1970-01-01", tz = "UTC")))
  }
  if (is.character(x)) {
    suppressWarnings({
      dt <- ymd_hms(x, quiet = TRUE, tz = "UTC")
      idx <- is.na(dt)
      if (any(idx)) dt[idx] <- ymd(x[idx], quiet = TRUE)
      idx <- is.na(dt)
      if (any(idx)) dt[idx] <- mdy(x[idx], quiet = TRUE)
      idx <- is.na(dt)
      if (any(idx)) dt[idx] <- dmy(x[idx], quiet = TRUE)
    })
    return(as_date(dt))
  }
  as_date(as.character(x))
}

# Apply date coercion
if (exists("ejscreen_all") && "query_date" %in% names(ejscreen_all)) {
  ejscreen_all <- ejscreen_all %>% mutate(query_date = to_Date(query_date))
}
if (exists("other_list")) {
  other_list <- other_list %>%
    map(~ if ("query_date" %in% names(.x)) mutate(.x, query_date = to_Date(query_date)) else .x)
} else {
  other_list <- list()
}

# Merge all datasets
all_tools <- bind_rows(ejscreen_all, !!!other_list)

# Map tool codes to full names
all_tools <- all_tools %>%
  mutate(
    source_file = if_else(is.na(source_file) | source_file == "",
      coalesce(source_file, tool_name), source_file
    ),
    base_tool = sub("_\\d+$", "", source_file), # ejscreen_1 -> ejscreen
    tool_name = dplyr::recode(
      base_tool,
      ejscreen = "EJScreen",
      cejst    = "Climate and Economic Justice Screening Tool (CEJST)",
      crv      = "Climate Risk Viewer",
      fri      = "Future Risk Index (FRI)",
      eji      = "Environmental Justice Index (EJI)",
      .default = tool_name # keep whatever was there if not matched
    )
  ) %>%
  select(-base_tool)

# Drop empty placeholder columns
all_tools <- all_tools %>%
  dplyr::select(
  -dplyr::any_of(c("issn", "citation_url", "volume",
                   "issue", "start_page", "end_page"))
)

# Save combined dataset
saveRDS(all_tools, here("05_cleaned_datasets", "all_tools.rds"))
write_csv(all_tools, here("05_cleaned_datasets", "all_tools.csv"))

## Analysis

### Stratified random sampling

For sampling, I selected **10% of the full dataset** collected in **August 2025** (prior to the September 15 refresh, which was a slightly bigger sample). The total number of studies from our September refresh was 2,212.

-   **Studies dataset:** 2,170 total → 217 sampled

-   **Publishers dataset:** 387 total → 40 sampled

Sampling was **stratified by `tool_name`** to ensure each tool was proportionally represented in the study sample. Both the **study** and **publisher** subsets were then **double-coded** to calculate inter-rater agreement (IRA).

For the **publishers**, we used a simple random sample generated in Excel rather than R.

A helpful reference for this approach is *“Stratified Sampling in R: A Practical Guide with Base R and dplyr”* \[@sanderson2024\].

The code below shows the stratified sampling method used for the **studies dataset** only.

I considered two allocation approaches:

1.  **Proportional-only allocation** (222 total). Each tool receives a number of samples proportional to its share of the full dataset. This preserves representativeness, but smaller tools (e.g., CRV and FRI) end up with too few studies to analyze meaningfully

2.  **Proportional with a minimum floor** (222 total, MIN = 10). Every tool receives at least **10 studies** (or all available if fewer exist). Larger tools such as **EJScreen** and **CEJST** contribute slightly fewer cases to make room for smaller tools.

I chose the **minimum floor approach** because it ensures that smaller tools are sufficiently represented for meaningful analysis and coder agreement checks.

**Tradeoff:**

-   *Proportional-only*: best for a population-level snapshot but leaves small strata nearly invisible.

-   *Floor allocation*: best for balanced comparisons across tools.

The final distribution of the **double-coded sample** is shown below:

In [ ]:
set.seed(2025)

# Load the dataset
all_tools <- readRDS(here("05_cleaned_datasets", "all_tools.rds")) %>%
  janitor::clean_names() %>%
  mutate(
    year = as.integer(year),
    study_id = dplyr::row_number()
  )

# Parameters
TOTAL <- nrow(all_tools) # should be 2212
SAMPLE_TOTAL <- ceiling(TOTAL * 0.10) # 10% => 221 (but we did 217 based on an older data pull)
MIN_PER_TOOL <- 10 # floor per tool
EXPAND_TOTAL <- TRUE # expand total if needed

# --------------------------------------------------------------------
# Allocation with per-tool minimums, then proportional distribution
# --------------------------------------------------------------------
tool_counts <- all_tools %>%
  count(tool_name, name = "n_tool") %>%
  arrange(desc(n_tool))

# Floors (min 10, capped at availability)
floor_alloc <- tool_counts %>%
  mutate(floor = pmin(n_tool, MIN_PER_TOOL))

reserved <- sum(floor_alloc$floor)

if (reserved > SAMPLE_TOTAL) {
  if (EXPAND_TOTAL) {
    SAMPLE_TOTAL <- reserved
  } else {
    stop(
  "Minimum per-tool exceeds total sample. ",
  "Lower `min_per_tool` or allow expand."
)
  }
}

remaining <- SAMPLE_TOTAL - reserved

# Proportional extras after floors
alloc <- floor_alloc %>%
  mutate(capacity = pmax(0, n_tool - floor))

if (remaining > 0 && sum(alloc$capacity) > 0) {
  alloc <- alloc %>%
    mutate(
      weight     = capacity / sum(capacity),
      extra_raw  = remaining * weight,
      extra_base = floor(extra_raw),
      remainder  = extra_raw - extra_base
    )
  leftover <- remaining - sum(alloc$extra_base)
  alloc <- alloc %>%
    arrange(desc(remainder)) %>%
    mutate(extra = extra_base + if_else(row_number() <= leftover, 1L, 0L)) %>%
    arrange(desc(n_tool))
} else {
  alloc <- alloc %>% mutate(extra = 0L)
}

# Final allocation
alloc <- alloc %>%
  mutate(alloc = pmin(n_tool, floor + extra))

tool_alloc <- alloc %>%
  select(tool_name, n_tool, alloc)

stopifnot(sum(tool_alloc$alloc) == SAMPLE_TOTAL)

# --------------------------------------------------------------------
# Stratified random sample
# --------------------------------------------------------------------
orig_cols <- names(all_tools)

sample_primary <- all_tools %>%
  inner_join(tool_alloc, by = "tool_name") %>%
  group_by(tool_name) %>%
  group_modify(~ dplyr::slice_sample(.x, n = unique(.x$alloc))) %>%
  ungroup() %>%
  mutate(
    double_code   = TRUE,
    topic_cat     = NA_character_,
    tool_use_type = NA_character_
  ) %>%
  select(all_of(orig_cols), double_code, topic_cat, tool_use_type)

stopifnot(nrow(sample_primary) == SAMPLE_TOTAL)

# --------------------------------------------------------------------
# Save 05_cleaned_datasetss (to 05_cleaned_datasets/ folder)
# --------------------------------------------------------------------
sample_primary %>%
  count(tool_name) %>%
  arrange(desc(n)) %>%
  print()

# A tibble: 5 × 2
  tool_name                                               n
  <chr>                                               <int>
1 EJScreen                                              111
2 Climate and Economic Justice Screening Tool (CEJST)    75
3 Environmental Justice Index (EJI)                      19
4 Climate Risk Viewer                                    11
5 Future Risk Index (FRI)                                 6

To support manual coding, I created an initial **codebook** with draft topic and tool-use categories. I’ll continue refining it by adding clear **definitions**, **inclusion and exclusion criteria**, and **examples** as the coding scheme develops.

In [ ]:
# Starter codebook for manual coding
topic_levels <- c(
  "Air quality / pollution",
  "Health impacts",
  "Extreme heat / climate risk",
  "Transportation",
  "Land use / housing",
  "Energy / utilities",
  "Water / flooding",
  "Equity / policy & governance"
)

tool_use_levels <- c(
  "Applied (used for analysis/targeting)",
  "Evaluated (validated/benchmarked the tool)",
  "Critiqued (limitations/methodological critique)",
  "Mentioned/Context (used for background)"
)

codebook <- tibble::tibble(
  variable = c(
    rep("topic_cat", length(topic_levels)),
    rep("tool_use_type", length(tool_use_levels))
  ),
  code = c(topic_levels, tool_use_levels),
  definition = NA_character_, # fill in
  include_if = NA_character_, # fill in
  exclude_if = NA_character_, # fill in
  examples = NA_character_ # fill in small examples/keywords
)

readr::write_csv(codebook, here("05_cleaned_datasets", "codebook_topic_tool_use.csv"))

### Inter-rater agreement

Before running the analysis, I’ll import two datasets that my colleagues and I collaborated on. These datasets include our manual categorizations of studies and their publishers.

**1. Study-level data**

-   `all_studies_ira.csv`: A collaborator and I manually categorized publishers for about **10% of the sample**. This dataset is used to calculate **inter-rater agreement (IRA)** and identify where our categorizations differed or aligned.
-   `all_studies_finalized.csv`: This is the **final dataset** after we reviewed disagreements and reached consensus. We updated the **codebook** accordingly, and applied the final categories to the full dataset. I will use this version for analyzing relevant research questions.
    -   The corresponding codebook, **`codebook_studies.csv`**, is located in the `codebooks/` folder.
    -   It defines **four categories** of study classification like applied, mentioned/context, etc. for a total of **10 subcategories.**

**2. Publisher-level data**

-   `all_publishers_ira.csv`: Similar to the studies dataset, a collaborator and I manually categorized about **11% of publishers**. This subset is used to assess IRA.
-   `all_publishers_finalized.csv`: The **finalized version** after resolving disagreements and updating the codebook. I will use this version for analyzing relevant research questions.
    -   The corresponding codebook, `codebook_publishers.csv`, is in the `codebooks/` folder.
    -   It defines **10 publisher types**, ranging from *commercial academic publishers* to *grey literature platforms*.

The first step in this analysis is to calculate **inter-rater agreement (IRA)** at both the study and publisher levels using **Cohen’s kappa**, a statistical measure of rater agreement.

I chose this approach because relying on a single person to categorize studies or publishers can introduce bias. With two collaborators independently code each dataset, I can identify where our classifications **converge** and **diverge**, and quantify that level of agreement.

Cohen’s kappa adjusts for agreement that occurs **by chance** and produces a value between **–1 and +1**, where –1 indicates no agreement and +1 indicates almost perfect agreement. In practice, it functions like a **correlation coefficient** based on a cross-tabulation of each rater’s classifications.

As McHugh (2012) explains, *“the squaring of kappa translates conceptually to the amount of accuracy (i.e., the reverse of error) in the data due to congruence among the data collectors.”*

**Interpretation of Cohen’s kappa** \[@mchugh2012\]

| Value of Kappa | Level of Agreement | % of Data that are Reliable |
|:--------------:|:------------------:|:---------------------------:|
|     0–.20      |        None        |            0–4%             |
|    .21–.39     |      Minimal       |            4–15%            |
|    .40–.59     |        Weak        |           15–35%            |
|    .60–.79     |      Moderate      |           35–63%            |
|    .80–.90     |       Strong       |           64–81%            |
|    Above.90    |   Almost Perfect   |           82–100%           |

A good resource that I’m following for IRA is *Cohen’s Kappa in R: For Two Categorical Variables* \[@datanovia2025\]. Another is the *agreement package:* <https://github.com/jmgirard/agreement>.

#### 1. Publisher-level data

Cohen’s κ = 0.89 (95 % CI \[0.79, 0.99\], p \< 0.001) indicates **almost perfect agreement** between the two raters according to McHugh (2012). The raw percent agreement was 91 %.

Here’s how to read the results table:

| Metric | Value | Interpretation |
|---------------------|-------------------|--------------------------------|
| **Percent agreement** | `0.909` (≈ 91%) | Raw proportion of identical ratings — shows high agreement overall |
| **Unweighted Cohen’s κ** | `0.8935` | Agreement beyond chance |
| **Weighted Cohen’s κ** | `0.8602` | Takes into account partial agreement (useful for *ordinal* data — not critical here) |
| **ASE** (Asymptotic Std. Error) | `0.0505` | Measures uncertainty of κ estimate |
| **z** | `17.69` | Test statistic for H₀ : κ = 0 (no agreement beyond chance) |
| **p-value** | `4.78 × 10⁻⁷⁰` | Extremely small → reject H₀; the raters’ agreement is highly significant |
| **95% CI (Unweighted)** | 0.79 – 0.99 | Even the lower bound (0.79) indicates strong agreement |

**Note on sample size for inter-rater agreement** Although the full dataset (`all_publishers_ira.csv`) contains 387 records, only the first 44 rows were double-coded by both raters (11% of the sample).

The remaining rows include missing or unassigned categories for at least one rater. Because `table()` in R automatically excludes `NA` values when computing cross-tabulations, the Cohen’s Kappa and percent agreement calculations below are effectively based on those 44 complete cases.

So, this code can be reused with any dataset of similar structure without needing to hardcode a row range. The computation will always use only non-missing, valid pairs of rater classifications.

In [ ]:
# Import publisher file
all_publishers_ira <- readr::read_csv(
  here::here("02_processed_data", "all_publishers_ira.csv"),
  show_col_types = FALSE,
  na = c("", "NA", "N/A", "NULL")
)

# Enforce types
all_publishers_ira <- all_publishers_ira |>
  mutate(
    publisher        = as.character(publisher),
    n                = as.integer(n),
    pub_type_rater_1 = as.character(pub_type_rater_1),
    pub_type_rater_2 = as.character(pub_type_rater_2),
    description      = as.character(description)
  )

# Define category levels
kappa_levels <- c(
  "Commercial academic publishers",
  "University presses",
  "Open access publishers",
  "Preprint / working paper servers",
  "Government repositories",
  "Legal / policy databases",
  "University / institutional repositories",
  "Grey literature platforms",
  "Scholarly society / association publishers",
  "None/Unknown"
)

# Apply factor levels
all_publishers_ira <- all_publishers_ira |>
  mutate(
    pub_type_rater_1 = factor(pub_type_rater_1, levels = kappa_levels),
    pub_type_rater_2 = factor(pub_type_rater_2, levels = kappa_levels)
  )

# Use to check for the recoding results
# table(all_publishers_ira$pub_type_rater_1, useNA = "ifany")
# table(all_publishers_ira$pub_type_rater_2, useNA = "ifany")

# Build contingency table and compute Cohen's Kappa
# (Unweighted Kappa corresponds to Cohen’s kappa for two raters)
tab_10x10 <- table(
  Rater1 = all_publishers_ira$pub_type_rater_1,
  Rater2 = all_publishers_ira$pub_type_rater_2
)

kappa_res <- vcd::Kappa(tab_10x10)

kappa_res

            value     ASE     z  Pr(>|z|)
Unweighted 0.8935 0.05050 17.69 4.780e-70
Weighted   0.8602 0.07517 11.44 2.536e-30

            
Kappa              lwr      upr
  Unweighted 0.7944860 0.992439
  Weighted   0.7129097 1.000000

[1] 0.9090909

#### 2. Study-level data

For the **topic category coding**, Cohen’s κ = 0.32 (95 % CI \[0.24, 0.39\], *p* \< 0.001) indicates **minimal agreement** between the two raters according to McHugh (2012). The raw percent agreement was **39%**.

For the **tool-use type coding**, Cohen’s κ = 0.16 (95 % CI \[0.09, 0.22\], *p* \< 0.001) reflects **no agreement** between raters. The raw percent agreement was **32%**.

In [ ]:
# Import publisher file
sample_studies_ira <- readr::read_csv(
  here::here("02_processed_data", "double_code_sample_ira.csv"),
  show_col_types = FALSE,
  na = c("", "NA", "N/A", "NULL")
)

# Enforce types ONLY for IRA-relavent variables
sample_studies_ira <- sample_studies_ira %>%
  mutate(
    topic_cat_rater_1      = as.character(topic_cat_rater_1),
    topic_cat_rater_2      = as.character(topic_cat_rater_2),
    tool_use_type_rater_1  = as.character(tool_use_type_rater_1),
    tool_use_type_rater_2  = as.character(tool_use_type_rater_2)
  )

# Define factor levels
topic_levels <- c(
  "Air quality & pollution",
  "Health & well-being",
  "Climate hazards & risks",
  "Transit, mobility & transportation",
  "Land use, housing & built environment infrastructure",
  "Energy & decarbonization",
  "Water resources & flooding",
  "Natural environment, resources & ecosystems",
  "Governance, policy & institutional frameworks",
  "Equity and justice",
  "Economy, labor & development",
  "Community engagement & knowledge",
  "Other"
)

tool_use_levels <- c(
  "Applied",
  "Evaluated",
  "Critiqued",
  "Mentioned/Context",
  "None"
)

# Apply factor levels
sample_studies_ira <- sample_studies_ira %>%
  mutate(
    topic_cat_rater_1     = factor(topic_cat_rater_1, levels = topic_levels),
    topic_cat_rater_2     = factor(topic_cat_rater_2, levels = topic_levels),
    tool_use_type_rater_1 = factor(tool_use_type_rater_1, levels = tool_use_levels),
    tool_use_type_rater_2 = factor(tool_use_type_rater_2, levels = tool_use_levels)
  )

# Compute Cohen's kappa for TOPIC
tab_topic <- table(
  Rater1 = sample_studies_ira$topic_cat_rater_1,
  Rater2 = sample_studies_ira$topic_cat_rater_2
)
kappa_topic <- vcd::Kappa(tab_topic)
pa_topic <- mean(
  sample_studies_ira$topic_cat_rater_1 == sample_studies_ira$topic_cat_rater_2,
  na.rm = TRUE
)

# Compute Cohen's kappa for TOOL USE TYP
tab_tool <- table(
  Rater1 = sample_studies_ira$tool_use_type_rater_1,
  Rater2 = sample_studies_ira$tool_use_type_rater_2
)
kappa_tool <- vcd::Kappa(tab_tool)
pa_tool <- mean(
  sample_studies_ira$tool_use_type_rater_1 == sample_studies_ira$tool_use_type_rater_2,
  na.rm = TRUE
)

# Simple percent agreement for TOPIC
percent_agreement_topic <- mean(
  sample_studies_ira$topic_cat_rater_1 == sample_studies_ira$topic_cat_rater_2,
  na.rm = TRUE
)
percent_agreement_topic

[1] 0.3921569

[1] 0.3221154

            value     ASE     z  Pr(>|z|)
Unweighted 0.3179 0.03726 8.534 1.415e-17
Weighted   0.3338 0.05020 6.649 2.956e-11

            
Kappa              lwr       upr
  Unweighted 0.2449245 0.3909685
  Weighted   0.2353783 0.4321608

[1] 0.3921569

            value     ASE     z  Pr(>|z|)
Unweighted 0.1561 0.03340 4.673 2.966e-06
Weighted   0.2037 0.04214 4.834 1.336e-06

            
Kappa               lwr       upr
  Unweighted 0.09063148 0.2215718
  Weighted   0.12112305 0.2863082

[1] 0.3221154

# A tibble: 2 × 4
  variable      kappa_unweighted kappa_weighted percent_agreement
  <chr>                    <dbl>          <dbl>             <dbl>
1 topic_cat                0.318          0.334             0.392
2 tool_use_type            0.156          0.204             0.322

### Research questions

Before analyzing each research question, I will merge the publisher and study datasets.

In [ ]:
# Import the all studies finalized file
all_studies_finalized <- readr::read_csv(
  here::here("02_processed_data", "all_studies_finalized.csv"),
  show_col_types = FALSE,
  na = c("", "NA", "N/A", "NULL")
)

out_clean <- here::here("05_cleaned_datasets")
out_viz <- here::here("06_viz_ready")

saveRDS(all_studies_finalized, file.path(out_clean, "all_studies_finalized.rds"))

# Load finalized publishers
pub_path <- here::here("05_cleaned_datasets", "all_publishers_finalized.rds")
all_publishers_finalized <- readRDS(pub_path)

# Create publisher lookup
pub_lookup <- all_publishers_finalized %>%
  mutate(
    publisher = as.character(publisher),
    pub_type_rater_1 = as.character(pub_type_rater_1),
    description = as.character(description)
  ) %>%
  filter(!is.na(publisher), publisher != "") %>%
  distinct(publisher, pub_type_rater_1, description)

# Enforce key types for studies
all_studies_finalized <- all_studies_finalized %>%
  mutate(
    publisher        = as.character(publisher),
    cites            = as.integer(cites),
    year             = as.integer(year),
    gs_rank          = as.integer(gs_rank),
    ecc              = as.integer(ecc),
    cites_per_year   = as.numeric(cites_per_year),
    cites_per_author = as.integer(cites_per_author),
    author_count     = as.integer(author_count),
    age              = as.integer(age),
    study_id         = as.integer(study_id),
    query_date       = if (inherits(query_date, "Date")) query_date else as_date(ymd(query_date))
  )

# Merge publisher data into studies
all_studies_merged <- all_studies_finalized %>%
  left_join(pub_lookup, by = "publisher") %>%
  rename(
    publisher_type        = pub_type_rater_1,
    publisher_description = description
  ) %>%
  mutate(
    publisher_type        = as.character(publisher_type),
    publisher_description = as.character(publisher_description)
  )

# Replace missing publisher_type with "None/Unknown"
all_studies_merged <- all_studies_merged |>
  mutate(
    publisher_type = if_else(
      is.na(publisher_type) | publisher_type == "",
      "None/Unknown",
      publisher_type
    )
  )

# Remove studies with "None" tool_use_type (per codebook) because
# we are not analyzing studies that have no relationships to the EJ tools
# Exclude false positives / not-in-scope studies (See 03_codebooks/codebook_studies.* for definitions)
all_studies_merged <- all_studies_merged |>
  mutate(tool_use_type = as.character(tool_use_type)) |>
  filter(
    !is.na(tool_use_type),
    stringr::str_trim(tool_use_type) != "None"
  )

# Save merged dataset
readr::write_csv(all_studies_merged, file.path("05_cleaned_datasets", "all_studies_merged.csv"))
saveRDS(all_studies_merged, file.path("05_cleaned_datasets", "all_studies_merged.rds"))
readr::write_csv(all_studies_merged, file.path(out_viz, "all_studies_merged.csv"))

#### Research question 1

*How many research papers or studies have cited or relied on the five U.S. federal climate and environmental justice tools that are threatened, not being maintained, or no longer publicly available?*

I will address each research question sequentially, cleaning and organizing the data throughout the analysis process. During this stage, I will prepare the data for exploration and export intermediate results as tidy CSV files for visualization and further analysis.

##### Total number of studies

In [ ]:
nrow(all_studies_merged)

[1] 1826

##### Count of studies per tool

In [ ]:
df_tool <- all_studies_merged |>
  count(tool_name, name = "n_studies") |>
  arrange(desc(n_studies))

hchart(
  df_tool,
  "column",
  hcaes(x = tool_name, y = n_studies)
) |>
  hc_title(text = "Number of studies by tool") |>
  hc_subtitle(
    text = "Total count of research papers citing or relying on each tool"
  ) |>
  hc_xAxis(title = list(text = "Tool")) |>
  hc_yAxis(
    title = list(text = "Number of studies"),
    allowDecimals = FALSE
  ) |>
  hc_tooltip(pointFormat = "<b>{point.y}</b> studies") |>
  hc_legend(enabled = FALSE) |>
  hc_exporting(enabled = TRUE)

##### Count of studies per year per tool

In [ ]:
df_year_tool <- all_studies_merged |>
  filter(!is.na(year)) |>
  count(tool_name, year, name = "n_studies") |>
  arrange(tool_name, year)

hchart(
  df_year_tool,
  "line",
  hcaes(x = year, y = n_studies, group = tool_name)
) |>
  hc_title(text = "Studies over time by tool") |>
  hc_subtitle(
    text = "Annual counts of studies citing or relying on each tool"
  ) |>
  hc_xAxis(title = list(text = "Year"), allowDecimals = FALSE) |>
  hc_yAxis(
    title = list(text = "Number of studies"),
    allowDecimals = FALSE
  ) |>
  hc_tooltip(
    shared = TRUE,
    crosshairs = TRUE,
    headerFormat = "<b>Year: {point.key}</b><br/>",
    pointFormat  = "{series.name}: <b>{point.y}</b><br/>"
  ) |>
  hc_legend(title = list(text = "Tool")) |>
  hc_exporting(enabled = TRUE)

##### Top 10 most cited studies overall per tool

In [ ]:
df_top_cites <- all_studies_merged |>
  filter(!is.na(cites)) |>
  group_by(tool_name) |>
  slice_max(order_by = cites, n = 10, with_ties = FALSE) |>
  ungroup() |>
  select(tool_name, year, title, cites, authors) |>
  arrange(tool_name, desc(cites))

df_top_cites

# A tibble: 34 × 5
   tool_name                                            year title cites authors
   <chr>                                               <int> <chr> <int> <chr>  
 1 Climate Risk Viewer                                    NA USDA…    27 I Fore…
 2 Climate Risk Viewer                                    NA Fore…     0 A Duga…
 3 Climate Risk Viewer                                  2025 Fore…     0 PW Cla…
 4 Climate and Economic Justice Screening Tool (CEJST)  2023 Metr…    73 E Bake…
 5 Climate and Economic Justice Screening Tool (CEJST)  2023 Char…    68 PGT Le…
 6 Climate and Economic Justice Screening Tool (CEJST)  2022 Resi…    64 M Manw…
 7 Climate and Economic Justice Screening Tool (CEJST)  2023 Air …    62 Y Wang…
 8 Climate and Economic Justice Screening Tool (CEJST)  2022 Mapp…    59 D Raim…
 9 Climate and Economic Justice Screening Tool (CEJST)  2023 Eval…    45 S Kime…
10 Climate and Economic Justice Screening Tool (CEJST)  2024 Meas…    43 JA Cas…
# ℹ 24 mo

##### Top 10 highest `cites_per_year` per tool

This shows which studies are “most cited relative to their age.”

In [ ]:
df_top_cpy <- all_studies_merged |>
  filter(!is.na(cites_per_year)) |>
  group_by(tool_name) |>
  slice_max(order_by = cites_per_year, n = 10, with_ties = FALSE) |>
  ungroup()

df_top_cpy |>
  select(tool_name, year, title, cites_per_year, cites, authors) |>
  arrange(tool_name, desc(cites_per_year))

# A tibble: 34 × 6
   tool_name                             year title cites_per_year cites authors
   <chr>                                <int> <chr>          <dbl> <int> <chr>  
 1 Climate Risk Viewer                     NA Fore…            0       0 A Duga…
 2 Climate Risk Viewer                     NA USDA…            0      27 I Fore…
 3 Climate Risk Viewer                   2025 Fore…            0       0 PW Cla…
 4 Climate and Economic Justice Screen…  2024 Meas…           43      43 JA Cas…
 5 Climate and Economic Justice Screen…  2024 Rece…           41      41 L Cain…
 6 Climate and Economic Justice Screen…  2023 Metr…           36.5    73 E Bake…
 7 Climate and Economic Justice Screen…  2024 Elec…           36      36 GJ Car…
 8 Climate and Economic Justice Screen…  2023 Char…           34      68 PGT Le…
 9 Climate and Economic Justice Screen…  2024 Heal…           31      31 T Xue,…
10 Climate and Economic Justice Screen…  2023 Air …           31      62 Y Wang…
# ℹ 24 mo

##### Average `cites_per_year` over time per tool

This shows whether more recent papers citing a tool are getting traction.

In [ ]:
df_avg_cpy <- all_studies_merged |>
  filter(!is.na(year), !is.na(cites_per_year)) |>
  group_by(tool_name, year) |>
  summarise(
    avg_cpy = mean(cites_per_year, na.rm = TRUE),
    .groups = "drop"
  )

hchart(
  df_avg_cpy,
  "line",
  hcaes(x = year, y = avg_cpy, group = tool_name)
) |>
  hc_title(text = "Average cites per year over time, by tool") |>
  hc_yAxis(title = list(text = "Average cites per year")) |>
  hc_exporting(enabled = TRUE)

##### Publishers

Which top 10 publishers/platforms appear most frequently?

In [ ]:
df_10_publishers <- all_studies_merged |>
  filter(publisher != "") |>
  count(publisher, sort = TRUE) |>
  slice_head(n = 10)

df_10_publishers

# A tibble: 10 × 2
   publisher                n
   <chr>                <int>
 1 search.proquest.com    159
 2 osti.gov               116
 3 HeinOnline             115
 4 Elsevier                94
 5 Springer                69
 6 papers.ssrn.com         50
 7 liebertpub.com          47
 8 Taylor &Francis         46
 9 Wiley Online Library    40
10 rosap.ntl.bts.gov       34

List all publishers from the sample.

In [ ]:
df_all_publishers <- all_studies_merged |>
  filter(publisher != "") |>
  count(publisher, sort = TRUE)

write_csv(df_all_publishers, here("06_viz_ready", "all_publishers.csv"))

#### Research question 2

*What types of research topics occur most frequently overall and within each tool, and how do these topics vary by citation frequency?*

For this analysis, I’ll use the combined dataset **`all_studies_merged.csv`**. This file already contains all the information I need to explore how research topics align with specific tools and how often each study is cited.

The plan is to create a **bubble chart** to visualize the relationship between:

-   The *tool name* (x-axis),

-   The *research topic category* (y-axis), and

-   The *citation frequency* (bubble size).

No additional CSVs are needed for this step.

#### Research question 3

*In what ways are these tools used in research studies overall and within each tool, and how does their use vary by citation frequency?*

This analysis follows the **same logic as Question 2**, using the **`all_studies_merged.csv`** dataset. Here, I’ll focus on categorizing how tools are *applied, mentioned, evaluated, and critiqued*.

## Next Step

And voilà! That completes the analysis portion 🎉

Next, I’ll take the cleaned CSVs exported in the **`06_viz_ready/`** folder and **`all_studies_merged.csv`** to move into the visualization stage. While I often use **{highcharter}** and **Apache ECharts**, I’ll likely use **Flourish** for this project so that my team at **PEDP** can easily update the visualizations if new data is added later.

Flourish is a great **low-code/no-code tool**, which perfect for quick collaboration and accessible storytelling.